## Data Manipulation

In [3]:
# import statements

import os
import io
import pandas as pd
import numpy as np
import difflib
from datetime import datetime, timedelta

import gspread
from df2gspread import df2gspread as d2g
from df2gspread import gspread2df as g2d
from oauth2client.service_account import ServiceAccountCredentials

### Get DataFrames

In [4]:
scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']
json_file_name = os.getcwd() + "/sportsbetting-376321-f2ada03a7020.json"
credentials = ServiceAccountCredentials.from_json_keyfile_name(json_file_name, scope)
gc = gspread.authorize(credentials)
spreadsheet_key = '1qZfM3myJ9naCc_AT7U20Zjv5UPMEa20sSTVuG7HlNNc'

In [214]:
# Betting Line Data

wks_name = "BettingPros"
betting_df = g2d.download(gfile=spreadsheet_key, wks_name=wks_name, col_names=True, row_names=True, credentials=credentials)
betting_df = betting_df.reset_index()
betting_df = betting_df.rename(columns={"index": "Player"})
print(betting_df.shape)
betting_df.head(5)

(4609, 3)


,Player,Line,Date
0,Jayson Tatum,22.5,2022-10-18
1,Stephen Curry,23.5,2022-10-18
2,De'Anthony Melton,7.5,2022-10-18
3,Anthony Davis,24.5,2022-10-18
4,Andrew Wiggins,15.5,2022-10-18


### Manipulate DataFrame

In [190]:
# Player Game Data 

wks_name = "PlayerData"
player_game_data_df = g2d.download(gfile=spreadsheet_key, wks_name=wks_name, col_names=True, row_names=True, credentials=credentials)

columns = ['Player', 'Date', 'PTS', 'Age', 'Team', 'Opp', 'MP', 'FG', 'FGA', '2P', '2PA', '3P', '3PA', 
           'FT', 'FTA', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'BPM']
player_game_data_df = player_game_data_df[columns]
player_game_data_df = player_game_data_df.sort_values(by=['Date'], ignore_index=True)
player_game_data_df['GP'] = 1

player_game_data_df.head(5)
stats = ['PTS', 'MP', 'FG', 'FGA', '2P', '2PA', '3P', '3PA', 'FT', 'FTA', 'ORB', 'DRB', 
         'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'BPM', 'GP']

for index1, row1 in player_game_data_df.iterrows():  
    for index2 in range(index1-1, -1, -1):
        row2 = player_game_data_df.loc[index2]
        if row2['Player'] == row1['Player']:
            for stat in stats:
                player_game_data_df.at[index1, stat] = float(row1[stat]) + float(row2[stat])
            break

player_game_data_df

,Player,Date,PTS,Age,Team,Opp,MP,FG,FGA,2P,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,BPM,GP
0,Kevon Looney,2022-10-18,7,26-254,GSW,LAL,21,3,4,3,...,2,4,6,2,1,0,0,2,4.9,1
1,Ryan Rollins,2022-10-18,0,20-107,GSW,LAL,2,0,1,0,...,0,1,1,1,0,0,0,2,-21.6,1
2,Anthony Davis,2022-10-18,27,29-221,LAL,GSW,36,10,22,10,...,0,6,6,0,4,1,3,2,-0.1,1
3,Montrezl Harrell,2022-10-18,2,28-265,PHI,BOS,11,1,3,1,...,0,0,0,0,0,1,1,3,-16.4,1
4,Kendrick Nunn,2022-10-18,13,27-076,LAL,GSW,23,5,9,2,...,0,3,3,2,1,0,3,2,1.3,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15969,Amir Coffey,2023-01-29,149,25-226,LAC,CLE,539,47,126,37,...,16,32,48,48,7,3,15,48,-197.3,38
15970,Mason Plumlee,2023-01-29,622,32-330,CHO,MIA,1436,240,356,240,...,171,325,496,183,32,33,79,145,123,51
15971,Lamar Stevens,2023-01-29,201,25-204,CLE,LAC,699,79,175,61,...,23,94,117,21,14,14,19,51,-133.7,36
15972,Evan Mobley,2023-01-29,760,21-225,CLE,LAC,1710,311,561,299,...,120,325,445,133,36,67,106,141,43.5,50


In [144]:
# Team Game Data

wks_name = "TeamData"
team_game_data_df = g2d.download(gfile=spreadsheet_key, wks_name=wks_name, col_names=True, row_names=True, credentials=credentials)

team1_stats = ['Team', 'Opp', 'Date', 'FG', 'FGA', '2P',
               '2PA', '3P', '3PA', 'FT', 'FTA', 'PTS.1']
               
team2_stats = ['Opp', 'Team', 'Date', 'FG.1','FGA.1', '2P.1', 
               '2PA.1', '3P.1', '3PA.1', 'FT.1', 'FTA.1', 'PTS.2']

stats = ['GP', 'FG', 'FGA', '2P', '2PA', '3P', '3PA', 'FT', 'FTA', 'PTS']

team1_game_data_df = team_game_data_df[team1_stats]
team1_game_data_df['GP'] = 1
team1_game_data_df = team1_game_data_df.rename(columns={"PTS.1": "PTS"})

team2_game_data_df = team_game_data_df[team2_stats]
team2_game_data_df['GP'] = 1
team2_game_data_df = team2_game_data_df.rename(columns={"Opp": "Team",
                                                        "Team": "Opp",
                                                        "FG.1": "FG",
                                                        "FGA.1": "FGA",
                                                        "2P.1": "2P",
                                                        "2PA.1": "2PA",
                                                        "3P.1": "3P",
                                                        "3PA.1": "3PA",
                                                        "FT.1": "FT",
                                                        "FTA.1": "FTA",
                                                        "PTS.2": "PTS"})

team_game_data_df = team1_game_data_df.append(team2_game_data_df)
team_game_data_df = team_game_data_df.sort_values(by=['Date'], ignore_index=True)
team_game_data_df = team_game_data_df.drop_duplicates(ignore_index=True)

for index1, row1 in team_game_data_df.iterrows():  
    for index2 in range(index1-1, -1, -1):
        row2 = team_game_data_df.loc[index2]
        if row2['Team'] == row1['Team']:
            for stat in stats:
                team_game_data_df.at[index1, stat] = float(row1[stat]) + float(row2[stat])
            break

team_game_data_df

<ipython-input-144-eedd13556baf>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  team1_game_data_df['GP'] = 1
<ipython-input-144-eedd13556baf>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  team2_game_data_df['GP'] = 1


,Team,Opp,Date,FG,FGA,2P,2PA,3P,3PA,FT,FTA,PTS,GP
0,PHI,BOS,2022-10-18,40,80,27,46,13,34,24,28,117,1
1,LAL,GSW,2022-10-18,40,94,30,54,10,40,19,25,109,1
2,GSW,LAL,2022-10-18,45,99,29,54,16,45,17,23,123,1
3,BOS,PHI,2022-10-18,46,82,34,47,12,35,22,28,126,1
4,MIN,OKC,2022-10-19,39,94,29,56,10,38,27,33,115,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1505,CLE,LAC,2023-01-29,2123,4388,1521,2749,602,1639,951,1224,5799,52
1506,NOP,MIL,2023-01-29,2148,4509,1586,2935,562,1574,1007,1294,5865,51
1507,MEM,IND,2023-01-29,2180,4651,1612,3002,568,1649,905,1267,5833,50
1508,MIL,NOP,2023-01-29,2075,4491,1360,2509,715,1982,838,1138,5703,50


### Combine DataFrames

In [213]:
# Combine Data

df = pd.merge(player_game_data_df, team_game_data_df, 
         left_on = ['Team', 'Date'], right_on = ['Opp', 'Date'], 
         how = 'left')
df = df.dropna()

df = df.astype({'PTS_x': 'int32'})
df = df.astype({'MP': 'int32'})
df = df.astype({'GP_x': 'int32'})
df = df.astype({'FG_x': 'int32'})
df = df.astype({'FGA_x': 'int32'})
df = df.astype({'2P_x': 'int32'})
df = df.astype({'2PA_x': 'int32'})
df = df.astype({'3P_x': 'int32'})
df = df.astype({'3PA_x': 'int32'})
df = df.astype({'FT_x': 'int32'})
df = df.astype({'FTA_x': 'int32'})

df = df.astype({'GP_y': 'int32'})
df = df.astype({'FG_y': 'int32'})
df = df.astype({'FGA_y': 'int32'})
df = df.astype({'2P_y': 'int32'})
df = df.astype({'2PA_y': 'int32'})
df = df.astype({'3P_y': 'int32'})
df = df.astype({'3PA_y': 'int32'})
df = df.astype({'FT_y': 'int32'})
df = df.astype({'FTA_y': 'int32'})

df["PPG"] = df["PTS_x"] / df["GP_x"]
df["MP"] = df["MP"] / df["GP_x"]
df["FG"] = df["FG_x"] / df["GP_x"]
df["FGA"] = df["FGA_x"] / df["GP_x"]
df["FG%"] = df["FG_x"] / df["FGA_x"]
df['2P'] = df["2P_x"] / df["GP_x"]
df['2PA'] = df["2PA_x"] / df["GP_x"]
df['2P%'] = df["2P_x"] / df["2PA_x"]
df['3P'] = df["3P_x"] / df["GP_x"]
df['3PA'] = df["3PA_x"] / df["GP_x"]
df['3P%'] = df["3P_x"] / df["3PA_x"]
df['FT'] = df["FT_x"] / df["GP_x"]
df['FTA'] = df["FTA_x"] / df["GP_x"]
df['FT%'] = df["FT_x"] / df["FTA_x"]
df['GP'] = df["GP_x"]
df['FG_opp'] = df["FG_y"] / df["GP_y"]
df['FGA_opp'] = df["FGA_y"] / df["GP_y"]
df['FG%_opp'] = df["FG_y"] / df["FGA_y"]
df['2P_opp'] = df["2P_y"] / df["GP_y"]
df['2PA_opp'] = df["2PA_y"] / df["GP_y"]
df['2P%_opp'] = df["2P_y"] / df["2PA_y"]
df['3P_opp'] = df["3P_y"] / df["GP_y"]
df['3PA_opp'] = df["3PA_y"] / df["GP_y"]
df['3P%_opp'] = df["3P_y"] / df["3PA_y"]
df['FT_opp'] = df["FT_y"] / df["GP_y"]
df['FTA_opp'] = df["FTA_y"] / df["GP_y"]
df['FT%_opp'] = df["FT_y"] / df["FTA_y"]

columns_to_keep = ['Player', 'Date','PPG', 'FG', 'FGA', 'FG%', '2P', '2PA',
       '2P%', '3P', '3PA', '3P%', 'FT', 'FTA', 'FT%', 'GP', 'FG_opp',
       'FGA_opp', 'FG%_opp', '2P_opp', '2PA_opp', '2P%_opp', '3P_opp',
       '3PA_opp', '3P%_opp', 'FT_opp', 'FTA_opp', 'FT%_opp']

df = df[columns_to_keep]

all_dates = df['Date'].unique()
all_dates.sort
last_date = all_dates[-1]

for index1, row1 in df.iterrows():  
    if row1['Date'] == last_date:
        df.at[index1, 'Date'] = "XXXX-XX-XX"
    else:
        for index2, row2 in df[index1+1:].iterrows():
            if row1['Player'] == row2['Player']:
                df.at[index1, 'Date'] = row2['Date']
                break
df

,Player,Date,PPG,FG,FGA,FG%,2P,2PA,2P%,3P,...,FG%_opp,2P_opp,2PA_opp,2P%_opp,3P_opp,3PA_opp,3P%_opp,FT_opp,FTA_opp,FT%_opp
0,Kevon Looney,2022-10-21,7.000000,3.000000,4.000000,0.750000,3.000000,4.000000,0.750000,0.000000,...,0.425532,30.000000,54.000000,0.555556,10.000000,40.000000,0.250000,19.000000,25.000000,0.760000
1,Ryan Rollins,2022-10-23,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,...,0.425532,30.000000,54.000000,0.555556,10.000000,40.000000,0.250000,19.000000,25.000000,0.760000
2,Anthony Davis,2022-10-20,27.000000,10.000000,22.000000,0.454545,10.000000,19.000000,0.526316,0.000000,...,0.454545,29.000000,54.000000,0.537037,16.000000,45.000000,0.355556,17.000000,23.000000,0.739130
3,Montrezl Harrell,2022-10-20,2.000000,1.000000,3.000000,0.333333,1.000000,3.000000,0.333333,0.000000,...,0.560976,34.000000,47.000000,0.723404,12.000000,35.000000,0.342857,22.000000,28.000000,0.785714
4,Kendrick Nunn,2022-10-20,13.000000,5.000000,9.000000,0.555556,2.000000,3.000000,0.666667,3.000000,...,0.454545,29.000000,54.000000,0.537037,16.000000,45.000000,0.355556,17.000000,23.000000,0.739130
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15969,Amir Coffey,XXXX-XX-XX,3.921053,1.236842,3.315789,0.373016,0.973684,2.394737,0.406593,0.263158,...,0.483820,29.250000,52.865385,0.553292,11.576923,31.519231,0.367297,18.288462,23.538462,0.776961
15970,Mason Plumlee,XXXX-XX-XX,12.196078,4.705882,6.980392,0.674157,4.705882,6.980392,0.674157,0.000000,...,0.453496,27.509804,51.411765,0.535088,11.784314,35.235294,0.334446,18.254902,22.137255,0.824624
15971,Lamar Stevens,XXXX-XX-XX,5.583333,2.194444,4.861111,0.451429,1.694444,3.277778,0.516949,0.500000,...,0.469898,27.452830,52.000000,0.527939,12.603774,33.245283,0.379115,17.981132,23.358491,0.769790
15972,Evan Mobley,XXXX-XX-XX,15.200000,6.220000,11.220000,0.554367,5.980000,10.060000,0.594433,0.240000,...,0.469898,27.452830,52.000000,0.527939,12.603774,33.245283,0.379115,17.981132,23.358491,0.769790


In [222]:
df = pd.merge(betting_df,df, on=['Player','Date'])
df

,Player,Line,Date,PPG,FG,FGA,FG%,2P,2PA,2P%,...,FG%_opp,2P_opp,2PA_opp,2P%_opp,3P_opp,3PA_opp,3P%_opp,FT_opp,FTA_opp,FT%_opp
0,James Harden,21.5,2022-10-20,35.000000,9.000000,14.000000,0.642857,4.000000,5.000000,0.800000,...,0.560976,34.000000,47.000000,0.723404,12.000000,35.000000,0.342857,22.000000,28.000000,0.785714
1,Lonnie Walker IV,10.5,2022-10-20,5.000000,2.000000,7.000000,0.285714,2.000000,4.000000,0.500000,...,0.454545,29.000000,54.000000,0.537037,16.000000,45.000000,0.355556,17.000000,23.000000,0.739130
2,Tyrese Maxey,18.5,2022-10-20,21.000000,8.000000,16.000000,0.500000,6.000000,11.000000,0.545455,...,0.560976,34.000000,47.000000,0.723404,12.000000,35.000000,0.342857,22.000000,28.000000,0.785714
3,De'Anthony Melton,7.5,2022-10-20,5.000000,2.000000,4.000000,0.500000,1.000000,2.000000,0.500000,...,0.560976,34.000000,47.000000,0.723404,12.000000,35.000000,0.342857,22.000000,28.000000,0.785714
4,Montrezl Harrell,5.5,2022-10-20,2.000000,1.000000,3.000000,0.333333,1.000000,3.000000,0.333333,...,0.560976,34.000000,47.000000,0.723404,12.000000,35.000000,0.342857,22.000000,28.000000,0.785714
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4164,Isaac Okoro,7.5,2023-01-29,5.843137,2.098039,4.411765,0.475556,1.411765,2.352941,0.600000,...,0.468461,31.265306,59.122449,0.528823,12.081633,33.408163,0.361637,18.020408,22.693878,0.794065
4165,Terry Rozier,20.5,2023-01-29,21.475000,8.050000,19.500000,0.412821,5.350000,11.450000,0.467249,...,0.486698,32.125000,58.687500,0.547391,10.562500,29.020833,0.363963,18.291667,22.437500,0.815227
4166,Aaron Nesmith,9.5,2023-01-29,9.159091,3.136364,7.772727,0.403509,1.750000,3.750000,0.466667,...,0.460068,27.020408,50.122449,0.539088,14.244898,39.571429,0.359979,16.857143,22.816327,0.738819
4167,Larry Nance Jr.,7.5,2023-01-29,7.675000,3.250000,5.275000,0.616114,2.975000,4.550000,0.653846,...,0.478118,30.571429,55.244898,0.553380,11.122449,31.959184,0.348020,18.204082,23.469388,0.775652


### Write Combined Dataset to Google Sheets

In [228]:
wks_name = 'CombinedData'
d2g.upload(df, spreadsheet_key, wks_name, credentials=credentials, row_names=True)
print(df.shape)
df.head(5)

(4169, 29)


,Player,Line,Date,PPG,FG,FGA,FG%,2P,2PA,2P%,...,FG%_opp,2P_opp,2PA_opp,2P%_opp,3P_opp,3PA_opp,3P%_opp,FT_opp,FTA_opp,FT%_opp
0,James Harden,21.5,2022-10-20,35.0,9.0,14.0,0.642857,4.0,5.0,0.800000,...,0.560976,34.0,47.0,0.723404,12.0,35.0,0.342857,22.0,28.0,0.785714
1,Lonnie Walker IV,10.5,2022-10-20,5.0,2.0,7.0,0.285714,2.0,4.0,0.500000,...,0.454545,29.0,54.0,0.537037,16.0,45.0,0.355556,17.0,23.0,0.739130
2,Tyrese Maxey,18.5,2022-10-20,21.0,8.0,16.0,0.500000,6.0,11.0,0.545455,...,0.560976,34.0,47.0,0.723404,12.0,35.0,0.342857,22.0,28.0,0.785714
3,De'Anthony Melton,7.5,2022-10-20,5.0,2.0,4.0,0.500000,1.0,2.0,0.500000,...,0.560976,34.0,47.0,0.723404,12.0,35.0,0.342857,22.0,28.0,0.785714
4,Montrezl Harrell,5.5,2022-10-20,2.0,1.0,3.0,0.333333,1.0,3.0,0.333333,...,0.560976,34.0,47.0,0.723404,12.0,35.0,0.342857,22.0,28.0,0.785714
